<img src="http://oproject.org/tiki-download_file.php?fileId=8&display&x=450&y=128" width="200" height="200">
<img src="http://gfif.udea.edu.co/root/tmva/img/tmva_logo.gif" width="200" height="200">

# Autoencoder Variable Transformation
<hr style="border-top-width: 4px; border-top-color: #34609b;"> 

### Initialize DataLoader and Factory 

In [1]:
TMVA::Tools::Instance();
TFile *inputFile = TFile::Open( "../datasets/mydataset.root"); 
TFile* outputFile = TFile::Open( "mydataset_output.root", "RECREATE" );
TMVA::Factory *factory = new TMVA::Factory("TMVARegression", outputFile, 
                                           "!V:!Silent:Color:DrawProgressBar:AnalysisType=Regression" );
TMVA::DataLoader *loader=new TMVA::DataLoader("mydataset");

--- Factory                  : You are running ROOT Version: 6.07/07, Apr 1, 2016
--- Factory                  : 
--- Factory                  : _/_/_/_/_/ _|      _|  _|      _|    _|_|   
--- Factory                  :    _/      _|_|  _|_|  _|      _|  _|    _| 
--- Factory                  :   _/       _|  _|  _|  _|      _|  _|_|_|_| 
--- Factory                  :  _/        _|      _|    _|  _|    _|    _| 
--- Factory                  : _/         _|      _|      _|      _|    _| 
--- Factory                  : 
--- Factory                  : ___________TMVA Version 4.2.1, Feb 5, 2015
--- Factory                  : 


### Setup DataLoader

In [2]:
loader->AddVariable("var0", 'F');
loader->AddVariable("var1", 'F');
loader->AddVariable("var2", 'F');
loader->AddVariable("var3 := var0-var1", 'F');
loader->AddVariable("var4 := var0*var2", 'F');
loader->AddVariable("var5 := var1+var2", 'F');
TTree *tsignal = (TTree*)inputFile->Get("MyMCSig");
TTree *tbackground = (TTree*)inputFile->Get("MyMCBkg");
TCut mycuts = "";
TCut mycutb = "";
loader->AddSignalTree( tsignal,     1.0 );
loader->AddBackgroundTree( tbackground, 1.0 );
loader->PrepareTrainingAndTestTree( mycuts, mycutb,"nTrain_Signal=3000:nTrain_Background=3000:nTest_Signal=1449:nTest_Background=1449:SplitMode=Random:NormMode=NumEvents:!V");

--- DataSetInfo              : Dataset[mydataset] : Added class "Signal"	 with internal class number 0
--- mydataset                : Add Tree MyMCSig of type Signal with 5449 events
--- DataSetInfo              : Dataset[mydataset] : Added class "Background"	 with internal class number 1
--- mydataset                : Add Tree MyMCBkg of type Background with 5449 events
--- mydataset                : Preparing trees for training and testing...


### Get Dataset Info

In [3]:
loader->CalcNorm();

--- DataSetFactory           : Dataset[mydataset] : Splitmode is: "RANDOM" the mixmode is: "SAMEASSPLITMODE"
--- DataSetFactory           : Dataset[mydataset] : Create training and testing trees -- looping over class "Signal" ...
--- DataSetFactory           : Dataset[mydataset] : Weight expression for class 'Signal': ""
--- DataSetFactory           : Dataset[mydataset] : Create training and testing trees -- looping over class "Background" ...
--- DataSetFactory           : Dataset[mydataset] : Weight expression for class 'Background': ""
--- DataSetFactory           : Dataset[mydataset] : Number of events in input trees (after possible flattening of arrays):
--- DataSetFactory           : Dataset[mydataset] :     Signal          -- number of events       : 5449   / sum of weights: 5449 
--- DataSetFactory           : Dataset[mydataset] :     Background      -- number of events       : 5449   / sum of weights: 5449 
--- DataSetFactory           : Dataset[mydataset] :     Signal     tre

### Setup Deep Neural Network

In [4]:
TString layoutString ("Layout=TANH|3,LINEAR");
TString training0 ("LearningRate=1e-1,Momentum=0.0,Repetitions=1,ConvergenceSteps=300,BatchSize=20,TestRepetitions=15,WeightDecay=0.001,Regularization=NONE,DropConfig=0.0+0.5+0.5+0.5,DropRepetitions=1,Multithreading=True");
TString training1 ("LearningRate=1e-2,Momentum=0.5,Repetitions=1,ConvergenceSteps=300,BatchSize=30,TestRepetitions=7,WeightDecay=0.001,Regularization=L2,Multithreading=True,DropConfig=0.0+0.1+0.1+0.1,DropRepetitions=1");
// TString training2 ("LearningRate=1e-2,Momentum=0.3,Repetitions=1,ConvergenceSteps=300,BatchSize=40,TestRepetitions=7,WeightDecay=0.0001,Regularization=L2,Multithreading=True");
// TString training3 ("LearningRate=1e-3,Momentum=0.1,Repetitions=1,ConvergenceSteps=200,BatchSize=70,TestRepetitions=7,WeightDecay=0.0001,Regularization=NONE,Multithreading=True");

// TString training0 ("LearningRate=1e-5,Momentum=0.5,Repetitions=1,ConvergenceSteps=500,BatchSize=50,TestRepetitions=7,WeightDecay=0.01,Regularization=NONE,DropConfig=0.5+0.5+0.5+0.5,DropRepetitions=2,Multithreading=True");
// TString training1 ("LearningRate=1e-5,Momentum=0.9,Repetitions=1,ConvergenceSteps=170,BatchSize=30,TestRepetitions=7,WeightDecay=0.01,Regularization=L2,DropConfig=0.1+0.1+0.1,DropRepetitions=1,Multithreading=True");
// TString training2 ("LearningRate=1e-5,Momentum=0.3,Repetitions=1,ConvergenceSteps=150,BatchSize=40,TestRepetitions=7,WeightDecay=0.01,Regularization=NONE,Multithreading=True");
// TString training3 ("LearningRate=1e-6,Momentum=0.1,Repetitions=1,ConvergenceSteps=500,BatchSize=100,TestRepetitions=7,WeightDecay=0.0001,Regularization=NONE,Multithreading=True");

TString trainingStrategyString ("TrainingStrategy=");
// trainingStrategyString += training0 + "|" + training1 + "|" + training2 + "|" + training3;
trainingStrategyString += training0 + "|" + training1;
TString nnOptions ("AE(indexLayer=1;pretraining=false;!H:V:ErrorStrategy=SUMOFSQUARES:VarTransform=G:WeightInitialization=XAVIERUNIFORM");
nnOptions.Append (":");
nnOptions.Append (layoutString);
nnOptions.Append (":");
nnOptions.Append (trainingStrategyString);
nnOptions.Append (")");

In [ ]:
cout << nnOptions.Data() << endl;

AE(indexLayer=1;pretraining=false;!H:V:ErrorStrategy=SUMOFSQUARES:VarTransform=G:WeightInitialization=XAVIERUNIFORM:Layout=TANH|3,LINEAR:TrainingStrategy=LearningRate=1e-1,Momentum=0.0,Repetitions=1,ConvergenceSteps=300,BatchSize=20,TestRepetitions=15,WeightDecay=0.001,Regularization=NONE,DropConfig=0.0+0.5+0.5+0.5,DropRepetitions=1,Multithreading=True|LearningRate=1e-2,Momentum=0.5,Repetitions=1,ConvergenceSteps=300,BatchSize=30,TestRepetitions=7,WeightDecay=0.001,Regularization=L2,Multithreading=True,DropConfig=0.0+0.1+0.1+0.1,DropRepetitions=1)


### Apply Autoencoder Transform

In [ ]:
TMVA::DataLoader* newloader = loader->VarTransform(nnOptions);

--- mydataset                : [AE Transform] indexLayer = 1
--- mydataset                : [AE Transform] Pretraining = false
--- mydataset                : Preparing DataLoader for Autoencoder Transform DNN Training
--- DataSetInfo              : Dataset[temporary_loader_for_training] : Added class "Regression"	 with internal class number 0
--- temporary_loader_for_t...: Add Tree Dataset of type Regression with 6000 events
--- DataSetInfo              : Dataset[temporary_loader_for_training] : Class index : 0  name : Regression
--- DNN                      : Parsing option string: 
--- DNN                      : ... "!H:V:ErrorStrategy=SUMOFSQUARES:VarTransform=G:WeightInitialization=XAVIERUNIFORM:Layout=TANH|3,LINEAR:TrainingStrategy=LearningRate=1e-1,Momentum=0.0,Repetitions=1,ConvergenceSteps=300,BatchSize=20,TestRepetitions=15,WeightDecay=0.001,Regularization=NONE,DropConfig=0.0+0.5+0.5+0.5,DropRepetitions=1,Multithreading=True|LearningRate=1e-2,Momentum=0.5,Repetitions=1,Converg


learning rate reduced to 0.003125




llee
aalr
renlnaieirnanngrgi n nrirgana tgtre ea r trareete dedru uecrcdeeeudddc u etctdoeo  d t 00ot.. o000 00.0000.110099005510339111593335

1331
3


--- DNN                      : 
--- DNN                      : Training with learning rate = 0.01, momentum = 0.5, repetitions = 1
--- DNN                      : Drop configuration
--- DNN                      :     drop repetitions = 1
--- DNN                      :     Layer 0 = 0
--- DNN                      :     Layer 1 = 0.1
--- DNN                      :     Layer 2 = 0.1
--- DNN                      :     Layer 3 = 0.1
--- DNN                      : 



learning rate reduced to 0.005



learning rate reduced to 
learning rate reduced to 0.0002.50

0
l1le2ea5ar
rnniinngg  rraattee  rreedduucceedd  ttoo  0.0003125
0.0003125


--- DNN                      : 
--- DNN                      : Dataset[temporary_loader_for_training] : End of training                                              
--- DNN                      : Dataset[temporary_loader_for_training] : Elapsed time for training with 3000 events: 79.9 sec         
--- DNN                      : Dataset[temporary_loader_for_training] : Create MVA output for Dataset[temporary_loader_for_training] : regression on training sample
--- DNN                      : Dataset[temporary_loader_for_training] : Create results for training
--- DNN                      : Dataset[temporary_loader_for_training] : Evaluation of DNN on training sample
